In [1]:
import pandas as pd
from sqlalchemy import create_engine
from get_info import get_random_int, get_person_info, get_visit_occurrence, get_drug_exposure, get_condition_occurrence
import json

with open("/home/jovyan/work/.credentials.json", "r") as credential:
    credential = json.load(credential)
    url = credential["url"]

In [2]:
# DB 연결
engine = create_engine(url, connect_args={'options': '-csearch_path={}'.format('de')})

df = pd.read_sql_table(
    "clinical_note",
    con=engine
)

In [3]:
# CONTINUING 앞 부분만 가져오기
df["note_cut"] = df.apply(lambda row: row.str.split("CONTINUING")[0][0], axis=1)

In [4]:
# 정보 추출 후, list 안의 dictionary 형태로 저장
persons, visit_occurrences, drug_exposures, condition_occurrences = [], [], [], []
for note in df["note_cut"]:
    person_id, visit_occurrence_id, drug_exposure_id = get_random_int()
    persons.append(get_person_info(note, person_id))
    visit_occurrences.append(get_visit_occurrence(note, person_id, visit_occurrence_id))
    drug_exposures += get_drug_exposure(note, person_id, visit_occurrence_id, drug_exposure_id)
    condition_occurrences += get_condition_occurrence(note, person_id, visit_occurrence_id)

In [5]:
# dictionary를 Dataframe으로 변환
dicts_for_df = [persons, visit_occurrences, drug_exposures, condition_occurrences]
dfs = []
for dict_for_df in dicts_for_df:
    dfs.append(pd.DataFrame(dict_for_df))

In [ ]:
engine = create_engine(url, connect_args={'options': '-csearch_path={}'.format('walker105')})
tables = ["person", "visit_occurrence", "drug_exposure", "condition_occurrence"]
for i in range(len(dfs)):
    dfs[i].to_sql(tables[i], engine, if_exists='append', index=False)

In [7]:
dfs[2]

,drug_exposure_id,person_id,drug_exposure_start_date,drug_value,route_value,dose_value,unit_value,visit_occurrence_id
0,4227729,8573977,2011-06-20,Acetaminophen,Oral,325,MG,4006403
1,1198506,5250735,1984-06-07,FLUoxetine,Oral,20,MG,4990821


In [8]:
for note in df.note_cut:
    print(note)

Andrea7 Wolf938
Race:                White
Ethnicity:           Non-Hispanic
Gender:              M
Age:                 55
Birth Date:          1965-04-22
Marital Status:      M
--------------------------------------------------------------------------------
ALLERGIES:
No Known Allergies
--------------------------------------------------------------------------------
ENCOUNTER
2011-06-20 : Encounter at Cape Cod Vet Center : Encounter for Acute bronchitis (disorder)
Type: ambulatory
   
   MEDICATIONS:
  2011-06-20 : Acetaminophen 325 MG Oral Tablet for Acute bronchitis (disorder)
   
   CONDITIONS:
  2011-06-20 : Acute bronchitis (disorder)
   
   CARE PLANS:
  2011-06-20 : Respiratory therapy
                         Reason: Acute bronchitis (disorder)
                         Activity: Recommendation to avoid exercise
                         Activity: Deep breathing and coughing exercises
   
   REPORTS:
   
   OBSERVATIONS:
   
   PROCEDURES:
  2011-06-20 : Sputum examination (pro